In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from music21 import converter, midi, note, chord, pitch, instrument, stream
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense, Activation, Embedding, Flatten
import pickle
from keras.utils import to_categorical
import pickle

Using TensorFlow backend.


In [0]:
data = pd.read_csv("drive/My Drive/maestro-v2.0.0/maestro-v2.0.0.csv")

In [0]:
def get_notes_chords_rests(path):
    file = converter.parse(path)
    components = []
    for element in file.recurse():
        if isinstance(element, note.Note):
            components.append(str(element.pitch))
        elif isinstance(element, chord.Chord):
            components.append('.'.join(str(n) for n in element.normalOrder))
        elif isinstance(element, note.Rest):
            components.append("Rest")

    return components

In [5]:
a = []
count = 0
for i in range(10):
  print("{}% Done".format(i*10))
  a.append(get_notes_chords_rests("drive/My Drive/maestro-v2.0.0/" + data["midi_filename"][i]))

0% Done
10% Done
20% Done
30% Done
40% Done
50% Done
60% Done
70% Done
80% Done
90% Done


In [0]:
notes = [j for element in a for j in element]

In [0]:
def Sequences(notes): 
    seqL = 100
    unique_char = len(set(notes)) 
    pitchnames = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    network_input = []
    network_output = []
    for i in range(0, len(notes) - seqL):
        sequence_in = notes[i: i + seqL]
        sequence_out = notes[i + seqL]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])
    n_patterns = len(network_input)
    network_input = np.reshape(network_input, (n_patterns, seqL, 1))
    network_input = network_input / float(unique_char)
    network_output = to_categorical(network_output)
    return (network_input, network_output)

In [0]:
def Normal_LSTM_Model(network_in, unique_char): 
    model = Sequential()
    model.add(LSTM(128, input_shape=network_in.shape[1:], return_sequences=True))
    model.add(Dropout(0.2))
    model.add(LSTM(128, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(256))
    model.add(Dropout(0.3))
    model.add(Dense(unique_char))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ["accuracy"])

    return model 

In [0]:
from keras.callbacks import ModelCheckpoint     
def train_network(notes, epochs = 10):
    unique_char = len(set(notes))
    network_input, network_output = Sequences(notes) 
#     model = Normal_LSTM_Model(network_input, unique_char)
    model = load_model('drive/My Drive/Trained_70_Epochs.hdf5')
    print("Model loaded")
    print('Training in progress')
    filepath = 'drive/My Drive/Trained_70_Epochs.hdf5'  
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True)
    model.fit(network_input, network_output, epochs=epochs, batch_size=32, callbacks=[checkpoint]) 
    print('Training completed') 
    return model  

In [14]:
### Train the model 

model  = train_network(notes, 20)

Model loaded
Training in progress
Epoch 1/20
27064/27064 [==============================] - 316s 12ms/step - loss: 0.2082 - acc: 0.9420
Epoch 2/20
27064/27064 [==============================] - 317s 12ms/step - loss: 0.2129 - acc: 0.9411
Epoch 3/20
27064/27064 [==============================] - 316s 12ms/step - loss: 0.1915 - acc: 0.9475
Epoch 4/20
27064/27064 [==============================] - 315s 12ms/step - loss: 0.1801 - acc: 0.9500
Epoch 5/20
27064/27064 [==============================] - 317s 12ms/step - loss: 0.1985 - acc: 0.9458
Epoch 6/20
27064/27064 [==============================] - 314s 12ms/step - loss: 0.1784 - acc: 0.9507
Epoch 7/20
27064/27064 [==============================] - 316s 12ms/step - loss: 0.1785 - acc: 0.9527
Epoch 8/20
27064/27064 [==============================] - 315s 12ms/step - loss: 0.1728 - acc: 0.9519
Epoch 9/20
27064/27064 [==============================] - 315s 12ms/step - loss: 0.1630 - acc: 0.9547
Epoch 10/20
27064/27064 [=======================

In [0]:
def generate_notes(model, network_input, pitchnames, unique_char):
    start = np.random.randint(0, len(network_input)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern = network_input[start]
    prediction_output = []
    print('Generating notes........')
    i = 1
    for note_index in range(500):
        if note_index%50 ==0:
          print("{}% Done".format(i*10))
          i += 1
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(unique_char)
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)
        pattern = list(pattern)
        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    print('Notes Generated...')
    return prediction_output

In [16]:
unique_char = len(set(notes))
network_input, _ = Sequences(notes)
pitchnames = sorted(set(item for item in notes))
prediction_output = generate_notes(model, network_input, pitchnames, unique_char)

Generating notes........
10% Done
20% Done
30% Done
40% Done
50% Done
60% Done
70% Done
80% Done
90% Done
100% Done
Notes Generated...


In [0]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []
    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        elif pattern == "Rest":
            new_note = note.Rest(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)
    
    print('Saving Output file as midi....')

    midi_stream.write('midi', fp='/content/drive/My Drive/output70Epochs10Songs.mid')

In [19]:
create_midi(prediction_output)

Saving Output file as midi....
